In [ ]:
import datetime as dt
import os
import time
from pathlib import Path

import geopandas as gpd
import requests
from tqdm.notebook import tqdm

import seabeepy as sb
from seabeepy.config import SETTINGS

In [ ]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

# Add default licence

## 1. Update orthomosaics

Update mosaics published to GeoNode with `Licence` and `Licence link` attributes. Takes values from the config. file if provided, and otherwise assumes a default of `CC-BY-4`. 

In [ ]:
def find_folders_with_file(root_folder, target_file):
    matching_folders = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        if target_file in filenames:
            matching_folders.append(dirpath)

    return matching_folders


# Find all folders containing 'config.seabee.yaml'
root_dir = "/home/notebook/shared-seabee-ns9879k/seabirds"
target_file = "config.seabee.yaml"
mission_dirs = find_folders_with_file(root_dir, target_file)
len(mission_dirs)

In [ ]:
# Just the folders with valid configs.
valid_mission_dirs = [
    dir_path for dir_path in mission_dirs if sb.ortho.check_config_valid(dir_path)
]
len(valid_mission_dirs)

In [ ]:
# Update metadata abstract on GeoNode
for mission_dir in tqdm(valid_mission_dirs, desc="Processing missions"):
    try:
        layer_name = sb.ortho.get_layer_name(mission_dir)
        date = sb.ortho.parse_mission_data(mission_dir, parse_date=True)[2]
        abstract = sb.geo.get_html_abstract(str(mission_dir))
        metadata = {
            "abstract": abstract,
            "date": date.isoformat(),
            "date_type": "creation",
            "attribution": "SeaBee",
        }
        sb.geo.update_geonode_metadata(
            layer_name,
            SETTINGS.GEONODE_USER,
            SETTINGS.GEONODE_PASSWORD,
            metadata,
        )
        time.sleep(0.25)
    except AssertionError as e:
        # Not found on GeoNode (e.g. because mosaicing failed on NodeODM)
        pass

## 2. Update detections

In [ ]:
cnt = 0
for mission_dir in tqdm(valid_mission_dirs, desc="Processing detections"):
    try:
        mission_name = sb.ortho.get_layer_name(mission_dir)
        if os.path.exists(f"{mission_dir}/results"):
            res_dir = sb.ml.get_latest_results_dir(mission_dir)
            res_path = os.path.join(res_dir, "out.gpkg")
            if os.path.exists(res_path):
                gdf = gpd.read_file(res_path)
                ml_options = sb.ml.get_ml_options(mission_dir)
                if ml_options["task"] == "detection":
                    # Seabirds mission
                    layer_name = f"{mission_name}_detections"
                    gdf = sb.ml.convert_seabird_class_codes_to_names(
                        gdf, SETTINGS.DB_USER, SETTINGS.DB_PASSWORD
                    )
                    style_dict = {
                        layer_name: f"red_outline.sld",
                    }
                    gpkg_path = os.path.join(
                        mission_dir,
                        "results",
                        ml_options["task"],
                        ml_options["model"],
                        f"{layer_name}.gpkg",
                    )
                    metadata = {
                        "abstract": sb.geo.get_detection_abstract(
                            gdf, layer_name, ml_options["model"], str(gpkg_path)
                        ),
                        "date": dt.datetime.strptime(
                            gdf["datetimereg"].iloc[0], "%Y-%m-%d %H:%M:%S"
                        ).isoformat(),
                        "date_type": "creation",
                        "attribution": "SeaBee",
                    }
                    sb.geo.update_geonode_metadata(
                        layer_name,
                        SETTINGS.GEONODE_USER,
                        SETTINGS.GEONODE_PASSWORD,
                        metadata,
                    )
                    time.sleep(0.25)
                    cnt += 1
    except AssertionError as e:
        # Not found on GeoNode (e.g. because mosaicing failed on NodeODM)
        pass

print(cnt, "datasets updated.")